In [3]:
import sys
import os
import yaml
from pathlib import Path

try:
    if not(changed) :
        raise Exception("changed")
except:
    changed = True
    sys.path.append("../")
    os.chdir("../")

In [8]:
import numpy as np

# Put the names of the models to be aggregated here
models_list = [
    # "best_checkpoint_gin-default_20240121-222625.pt",
    # "best_checkpoint_qformer-scibert_20240121-182230.pt",
    # "best_checkpoint_gin-default_20240122-021420.pt",
    # # "best_checkpoint_gin-default_20240122-071447.pt",
    # "best_checkpoint_gin-default_20240122-025033.pt",
    # "best_checkpoint_gat-default_20240122-120034.pt",
    # "best_checkpoint_gin-default_20240122-043604.pt",
    # "best_checkpoint_gat-default_20240122-094517.pt",
    # "best_checkpoint_gin-default_20240122-171449.pt",
    # "best_checkpoint_gat-default_20240124-155615.pt",
    # "best_checkpoint_gat-default_20240124-114021.pt",
    # "best_checkpoint_gin-default_20240124-172833.pt",
    "best_checkpoint_gat-default_20240124-143533.pt",
    "best_checkpoint_gat-default_20240125-040049.pt",
    "best_checkpoint_gin-default_20240125-051258.pt",
    "best_checkpoint_gat-default_20240125-135114.pt",
    "best_checkpoint_gin-default_20240125-191811.pt",
    "best_checkpoint_gat-scibert_20240126-040152.pt",
    "best_checkpoint_gin-scibert_20240126-082746.pt",
]

mrrs = [
    # 0.61, 
    # 0.63, 
    # 0.52, 
    # # 0.56, 
    # 0.62, 
    # 0.64,
    # 0.63, 
    # 0.60, 
    # 0.60,
    # 0.68,
    # 0.68,
    # 0.68,
    0.78,
    0.78,
    0.78,
    0.82,
    0.77,
    0.75,
    0.74,
]

weights = np.array(mrrs) / np.sum(mrrs)

submissions_test = [
    x.replace(".pt", ".csv").replace("best_checkpoint_", "results_test_") for x in models_list
]

submissions_val = [
    x.replace(".pt", ".csv").replace("best_checkpoint_", "results_val_") for x in models_list
]

In [9]:
from src.trainer import BaseTrainer
from main import MODELS_DICT
import pandas as pd
#TODO keep job_id in the name of the submission

def get_aggregation(submissions):

    submissions_files = []
    for i, submission in enumerate(submissions):
        submissions_files.append(pd.read_csv("./runs/results/" + submission))

    aggregate_submission = submissions_files[0].copy()
    aggregate_submission = aggregate_submission * weights[0]
    for i in range(1, len(submissions_files)):
        aggregate_submission += submissions_files[i] * weights[i]
    aggregate_submission["ID"] = submissions_files[0]["ID"]

    submission_name = "ensemble_"
    submission_job_ids = []
    for i in range(len(submissions)):
        submission_name += "_".join(submissions[i].split("_")[2:-1]) + "_"
        submission_job_ids.append((submissions[i].split("_")[-1].replace(".csv", ""), weights[i]))

    return aggregate_submission, submission_name, submission_job_ids

from datetime import datetime
time_stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
aggregate_submission_test, submission_name_test, submission_job_ids = get_aggregation(submissions_test)

# aggregate_submission_test.to_csv(f"./runs/results/{submission_name_test}_{time_stamp}.csv", index=False)
# import json
# with open(f"./runs/results/{submission_name_test}_{time_stamp}.json", "w") as f:
#     json.dump(submission_job_ids, f)

aggregate_submission_val, submission_name_val, submission_job_ids = get_aggregation(submissions_val)

# from src.dataloader import GraphDataset
# gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]

# val_cids_dataset = GraphDataset(
#     root="./data/", gt=gt, split="val_cids"
# )

from sklearn.metrics import label_ranking_average_precision_score

cosine_similarity = aggregate_submission_val.values[:, 1:]
true_cids_ranking = np.eye(cosine_similarity.shape[0])

from sklearn.metrics import label_ranking_average_precision_score

mrr = label_ranking_average_precision_score(
    true_cids_ranking, cosine_similarity
)

print("MRR on validation set: {}".format(mrr))

MRR on validation set: 0.85411536798775
